In [92]:
from IPython.core.debugger import set_trace
import json
import pandas as pd
from tqdm.autonotebook import tqdm
tqdm.pandas()

import os
import sys
import re
import string
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

import django
django.setup()

from data.models import Newsdata
import glob

import spacy
nlp = spacy.load('en_core_web_lg')

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\tqdm\_tqdm.py:604: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [131]:
from django.http import JsonResponse

In [97]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = '((?: rt)|(?:^rt))[^ @]?'
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    PUNC = r'[{}]'.format(string.punctuation)
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    #text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), '', text.lower())
    text = text.lower()
    
    if remove_url:
        text = re.sub(URL, ' ', text)

    if remove_mention:
        text = re.sub(MENTION, ' ', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, ' ', text)
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS, PUNC)), ' ', text)
    return re.sub(LONG_BLANK, ' ', text).strip()

In [86]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS; len(stopwords)

326

In [64]:
fnames = glob.glob('newsdata/downloaded/*.json'); len(fnames)

7441

In [104]:
Newsdata.objects.all().delete()

(32, {'data.Newsdata': 32})

In [105]:
texts = []
for i, fname in enumerate(tqdm(fnames[:])):
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        doc = nlp(preproc(content['text']))
        
        obj = Newsdata()
        obj.id = i + 1
        obj.pub = content['pub']
        obj.authors = ','.join(content['authors'])
        obj.title = content['title']
        obj.published_at = pd.Timestamp(content['published_at'], tz='utc')
        obj.downloaded_at = pd.Timestamp(content['downloaded_at'], tz='utc')
        obj.text = content['text']
        obj.description = content['description']
        obj.top_image = content['top_image']
        obj.url = content['url']
        obj.words = ','.join([str(noun) for noun in doc.noun_chunks if str(noun) not in stopwords])
        obj.language = content['language']
        obj.save()

In [122]:
for obj in tqdm(Newsdata.objects.all()):
    obj.words = json.dumps(obj.words.split(','))
    obj.save()

In [139]:
[item for sublist in Newsdata.objects.all()[:3].values_list('words', flat=True) for item in json.loads(sublist)]

['an ant financial executive',
 'singapore',
 'the speed',
 'a patent',
 'businesses',
 'the intellectual property office',
 'singapore ipos',
 'wednesday aug',
 'an artificial intelligence',
 'patent',
 'chinese tech giant alibaba group',
 'a record time',
 'three months',
 'the patent',
 'ipos accelerated initiative',
 'artificial intelligence',
 'the average time',
 'such patents',
 'two to four years ipos',
 'the hope',
 'the expedited application',
 'process',
 'ai2 innovators',
 'their ai inventions',
 'the global market',
 'singapore',
 'a crucial development',
 'singapore',
 's bid',
 'the intellectual property ip hub',
 'edge technologies companies',
 'solutions',
 'this backdrop',
 'a patent',
 'critical benjamin bai vice president',
 'chief ip counsel',
 'singapore',
 'a pivotal role',
 'our entry',
 'markets',
 'our interest',
 'also singapore',
 'the world',
 'a trademark registration app',
 'just 10mins',
 'an application',
 'ipos advancement',
 'technology',
 'an exponen

In [141]:
now = pd.Timestamp.utcnow()
_from = (now - pd.DateOffset(days=1)).date()
news = Newsdata.objects.filter(published_at__date__gte=_from).order_by('-downloaded_at')
news.count()

2093

In [142]:
news[0].published_at

datetime.datetime(2019, 9, 3, 0, 0, tzinfo=<UTC>)

In [144]:
news[2092].published_at

datetime.datetime(2019, 9, 2, 0, 0, tzinfo=<UTC>)

In [62]:
for fname in tqdm(fnames):
    _remove = False
    
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        if content['pub']=='thinkprogress':
            _remove = True
            print(fname)
    
    #if _remove: os.remove(fname)